# Imports

In [1]:
import os
import cv2
import time
import shutil
import pandas as pd
import numpy as np
from PIL import Image
from PIL import Image as PilImage
from matplotlib import pyplot as plt

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Classic

In [3]:
def HairSegmenter(img, dilate):
    model_path = '/home/vuong.nguyen/vuong/augmentare/Bias Celeba/hair_segmenter.tflite'
    base_options = python.BaseOptions(model_asset_path=model_path)

    BaseOptions = mp.tasks.BaseOptions
    ImageSegmenter = mp.tasks.vision.ImageSegmenter
    ImageSegmenterOptions = mp.tasks.vision.ImageSegmenterOptions
    VisionRunningMode = mp.tasks.vision.RunningMode

    # Create a image segmenter instance with the image mode:
    options = ImageSegmenterOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.IMAGE,
        output_category_mask=True)

    BG_COLOR = (0, 0, 0) # gray
    MASK_COLOR = (255, 255, 255) # white

    with ImageSegmenter.create_from_options(options) as segmenter:
        # Perform image segmentation on the provided single image.
        # The image segmenter must be created with the image mode.
        segmented_masks = segmenter.segment(img)
        category_mask = segmented_masks.category_mask

        # Generate solid color images for showing the output segmentation mask.
        image_data = img.numpy_view()
        fg_image = np.zeros(image_data.shape, dtype=np.uint8)
        fg_image[:] = MASK_COLOR
        bg_image = np.zeros(image_data.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
        output_image = np.where(condition, fg_image, bg_image)
        mask = cv2.inRange(output_image, MASK_COLOR, MASK_COLOR)
        mask = np.clip(mask, 0, 1)

        kernel = np.ones((dilate, dilate), np.uint8)
        dilated_mask = cv2.dilate(mask, kernel)
    return mask, dilated_mask

In [6]:
def get_data_opencv(path_source, path_target, color_filter, alpha):
    files_list = sorted(os.listdir(path_source))
    for curr_file in files_list:
        face = cv2.imread(f"{path_source}/{curr_file}")
        image = mp.Image.create_from_file(f"{path_source}/{curr_file}")
        mask, _ = HairSegmenter(image, 5)
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        mask = (mask * 255).astype(np.uint8)
        _, mask = cv2.threshold(mask, thresh=180, maxval=255, type=cv2.THRESH_BINARY)

        hair_color = np.copy(face)
        # boolean indexing and assignment based on mask
        hair_color[(mask==255).all(-1)] = color_filter
        hair_color_w = cv2.addWeighted(hair_color, 1-alpha, face, alpha, 0)
        #hair_color_w = cv2.cvtColor(hair_color_w, cv2.COLOR_BGR2RGB)
        cv2.imwrite(f"{path_target}/new_{curr_file}", hair_color_w)

In [18]:
def get_df_data(path_target, label, sens):
    files_list = sorted(os.listdir(path_target))
    elements = []
    for curr_file in files_list:
        elements.append((curr_file, label, sens))
    df = pd.DataFrame(elements, columns=["image_id", "Male", "Sens"])
    return df

In [19]:
path_source = "/home/vuong.nguyen/vuong/augmentare/Bias Celeba/dataset_gan/Classic/female_blond"
path_target = "/home/vuong.nguyen/vuong/augmentare/Bias Celeba/new_dataset/Classic/female_gray"
color_filter = [91,91,91]
alpha = 0.408
get_data_opencv(path_source, path_target, color_filter, alpha)

In [23]:
path_target = "/home/vuong.nguyen/vuong/augmentare/Bias Celeba/new_dataset/Classic/male_blond"
df = get_df_data(path_target, 1, 2)

In [24]:
df.to_csv('/home/vuong.nguyen/vuong/augmentare/Bias Celeba/new_dataset/Classic/new_male_blond.csv', index=False)

# Copy files

In [10]:
def get_data(df, path_source, path_target):
    if len(df) < 1000:
        num = len(df)
    else:
        num = 1000

    #files_list = sorted(os.listdir(path_source))
    file_names = df["image_id"].sample(num).tolist()
    #file_names = files[:num]

    for curr_file in file_names:
        shutil.copy(os.path.join(path_source, curr_file), os.path.join(path_target, curr_file))

In [11]:
df_male_gray = pd.read_csv("/home/vuong.nguyen/vuong/augmentare/Bias Celeba/dataframe_dataset/df_male_gray.csv")

In [12]:
get_data(
    df = df_male_gray,
    path_source='dataset/img_align_celeba/img_align_celeba',
    path_target='dataset_gan/male_gray'
)